# Importamos librerías


In [15]:
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as Keras
Keras.clear_session() #Si había una sesión de keras abierto, la cierro para trabajar desde cero

# Parametros de la red neuronal

In [16]:
datos_entrenamiento = './Señales/Entrenamiento' #Directorios donde estan las imagenes
datos_validacion = './Señales/Prueba' #Directorios donde estan las imagenes
epocas = 5 #Numero de veces que va a iterar sobre todo el set de datos durante el entrenamiento
altura, longitud = 30, 30 #Tamaño al cual va a procesar las imagenes
tamaño_lote = 32 #Cantidad de imagenes a procesar a la vez
pasos = 1000 #Durante un paso, un lote es de imagenes es procesado. Indica la cantidad de veces que va a procesar cada lote en cada epoca.
pasos_validacion = 200 #Al final de cada epoca se corren 200 pasos con los set de datos de validacion
filtrosConv1 = 32 #El numero de filtros que aplicara en la primer convolución
filtrosConv2 = 64 #La profundidad de la imagen con una convolucion sera de 32 y con dos convoluciones sera de 64
tamaño_filtro1 = (3,3) #Tamaño del filtro que usa la primer convolucion. Altura 3 y longitud 3
tamaño_filtro2 = (2,2)
tamaño_pool = (2,2) #Tamaño del filtro del max pulling
clases = 60 #Numero de clases. Cantidad de carpetas con imagenes o set de imagenes

# Pre-procesamiento de imagenes

In [17]:
#Pre-procesamiento de imagenes para luego enviarlas a la red neuronal (Generador + transformacion de imagenes)
#Generador dice como va a preprocesar la informacion
entrenamiento_datagen = ImageDataGenerator( #Set de datos de entrenamiento
    rescale=1./255, #Cada uno de los pixeles tiene un rango de 0 a 255, en este caso todos los valores de pixeles tendran valores de 0 a 1
    shear_range=0.3, #Inclina la imagen para que el algoritmo aprenda a reconocer imagenes en todos los sentidos
    zoom_range=0.3, #Hace zoom para que aprenda a que no siempre aparecen "las señales completos"
    horizontal_flip = True #Toma una imagen y la invierte para que la red neuronal aprenda direccionalidad
)

validacion_datagen = ImageDataGenerator( #Set de datos de validacion
    rescale=1. / 255, # Cada uno de los pixeles tiene un rango de 0 a 255, en este caso todos los valores de pixeles tendran valores de 0 a 1
                      #Solo se las escala porque para validarlas no queremos hacerle zoom, girarla ni nada de eso
)

imagen_entrenamiento = entrenamiento_datagen.flow_from_directory( #Entra al directorio Señales/Entrenamiento, abre todas las carpetas, las procesa a altuna y longitud especificada
    #las procesa en un tamaño_lote de 32, y class_mode = 'categorical' es porque es una clasificacion categorica
    datos_entrenamiento,
    target_size=(altura, longitud),
    batch_size=tamaño_lote,
    class_mode='categorical'
)

imagen_validacion = validacion_datagen.flow_from_directory(
    datos_validacion,
    target_size=(altura, longitud),
    batch_size=tamaño_lote,
    class_mode='categorical'
)

Found 4284 images belonging to 60 classes.
Found 2411 images belonging to 60 classes.


# Creación de red CNN

In [18]:
cnn=Sequential() #La red que va a generar va a ser secuencial, es decir, son varias capas apiladas entre ellas

cnn.add(Convolution2D(filtrosConv1, tamaño_filtro1, padding='same', input_shape=(altura, longitud, 3), activation='relu')) #Añadimos la primer capa convolucional input_shape tiene altura, longitud y 3 canales (RGB)

cnn.add(MaxPooling2D(pool_size=tamaño_pool)) #Añado primer capa de maxpulling

cnn.add(Convolution2D(filtrosConv2, tamaño_filtro2, padding='same', activation='relu')) #Añado segunda capa convolucional

cnn.add(MaxPooling2D(pool_size=tamaño_pool)) #Añado segunda capa de maxpulling

#Empezamos clasificacion

cnn.add(Flatten()) #Hacemos a la imagen en una sola dimension. Aplana la informacion
cnn.add(Dense(265, activation='relu')) #Manda toda la info plana a una capa que tiene 265 capas
cnn.add(Dropout(0.5)) #Se apaga el 50% de las neuronas de Dense265 para evitar sobreajustar: ya que si todo el tiempo todas las neuronas estan activadas puede que las neuronas aprendan un camino especifico para clasificar señales,
#entonces, de manera aleatoria en cada paso solo activa el 50% de las neutoras para aprender caminos alternos
cnn.add(Dense(clases, activation='softmax')) #Softmax hace que te diga que probabilidades hay que sea cada una de las señales con un %. La que mas alto % tiene es la que muestra como precicción

cnn.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy']) #Compila el algoritmo, con parametros para optimizarlo:
#loss: durante el entrenamiento, su funcion de perdida (es decir que el algoritmo vea que tan bien o que tan mal va) va a ser categorical_crossentrpy
#optimizer: usamos Adam
#metrics: usamos accuracy

# Por ultimo para entrenar el algoritmo:

In [19]:
cnn.fit(imagen_entrenamiento, epochs=epocas, steps_per_epoch=pasos, validation_data=imagen_validacion, validation_steps=pasos_validacion) #Aqui le digo que va a entrenar la red neuronal con imagen_entrenamiento, le asigno los pasos, las epocas, etc

Epoch 1/5
1000/1000 [==============================] - 48s 48ms/step - loss: 1.1395 - accuracy: 0.6952 - val_loss: 0.3671 - val_accuracy: 0.9000
Epoch 2/5
1000/1000 [==============================] - 48s 48ms/step - loss: 0.3429 - accuracy: 0.8901 - val_loss: 0.2330 - val_accuracy: 0.9377
Epoch 3/5
1000/1000 [==============================] - 50s 50ms/step - loss: 0.2140 - accuracy: 0.9316 - val_loss: 0.2043 - val_accuracy: 0.9498
Epoch 4/5
1000/1000 [==============================] - 49s 49ms/step - loss: 0.1677 - accuracy: 0.9446 - val_loss: 0.1946 - val_accuracy: 0.9560
Epoch 5/5
1000/1000 [==============================] - 49s 49ms/step - loss: 0.1361 - accuracy: 0.9565 - val_loss: 0.2029 - val_accuracy: 0.9569


# Guardamos el modelo y los pesos en un archivo 

In [20]:
dir='./modelo/'
if not os.path.exists(dir): #Si no existe una carpeta que se llama modelo
    os.mkdir(dir) #Genera una carpeta que se llama modelo

cnn.save('./modelo/modelo.h5')  #Guarda la estructura del modelo
cnn.save_weights('./modelo/pesos.h5') #Guarda los pesos que tiene cada una de las capas que ya entreno